In [36]:
from pydantic import BaseModel
from openai import OpenAI
from IPython.display import display, Markdown
from collections import defaultdict
from enum import Enum
from PIL import Image as PILImage
from io import BytesIO
import itertools
import json
import uuid
import os
import base64
import re

from secret_vars import OPENAI_KEY

client = OpenAI(api_key=OPENAI_KEY)

In [20]:
DISPLAY_SPOILERS = True
GENERATE_THEMES_COUNT = 3
GENERATE_RUNES_COUNT = 20
COMFYUI_PATH = "D:\AI\Flux\ComfyUI"

## Theme generation

In [58]:
class RuneTheme(BaseModel):
    name: str
    description: str

    def __hash__(self):
        return hash((self.name, self.description))

class RuneThemes(BaseModel):
    ideas: list[RuneTheme]

def generate_themes() -> RuneThemes:
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": f"I'm making a game in which mundane tasks (such as throwing out trash or doing dishes) can be separated each into multiple runes. What overarching theme can I use for coming up with those runes? Reply with a list of {GENERATE_THEMES_COUNT} propositions."},
        ],
        response_format=RuneThemes,
    )
    return completion.choices[0].message.parsed

def display_themes(themes: RuneThemes):
    if not DISPLAY_SPOILERS:
        return
    display(Markdown("\n".join(f"## {theme.name}\n{theme.description}" for theme in themes.ideas)))

themes = generate_themes()
display_themes(themes)

## Nature's Cycle
Each rune represents a step in nature's cycle, emphasizing the importance of maintaining balance and harmony in the environment. For example, 'Seed' for initiation, 'Growth' for progression, and 'Harvest' for completion.
## Elemental Forces
Runes represent different elements (Earth, Water, Fire, Air) that relate to the task. 'Earth' for grounding tasks like organizing, 'Water' for fluid tasks like washing, 'Fire' for energizing tasks like motivating to declutter, and 'Air' for tasks related to lifting spirits.
## Daily Rhythm
Runes that embody the rhythm of daily life, such as 'Dawn' for starting a task, 'Noon' for peak productivity, 'Dusk' for winding down, and 'Midnight' for reflection and completion of chores.

## Rune generation

In [59]:
class Rune(BaseModel):
    name: str
    rune_description: str
    tasks_composed_of: str

    def stripped(self) -> str:
        return re.match("^Rune of(?: the)? (.+?)$", self.name).group(1)

    def encoded(self) -> str:
        return base64.b64encode(self.stripped().encode("ascii")).decode("ascii")

class Runes(BaseModel):
    ideas: list[Rune]

def generate_runes(themes: RuneThemes) -> Runes:
    non_duplicate_runes = defaultdict(list)
    taken_rune_names = set()
    
    i = 0
    while len(taken_rune_names) < GENERATE_RUNES_COUNT:
        theme = themes.ideas[i%len(themes.ideas)]
        
        required_rune_count = round((GENERATE_RUNES_COUNT-len(taken_rune_names))/max(1, len(themes.ideas)-i))
        if i >= len(themes.ideas):
            required_rune_count *= 2
        rune_count = required_rune_count+len(non_duplicate_runes[theme])
        runes = generate_runes_for_theme(theme, rune_count)

        for rune in runes.ideas:
            if rune.stripped() not in taken_rune_names:
                taken_rune_names.add(rune.stripped())
                non_duplicate_runes[theme].append(rune)
            if len(taken_rune_names) >= GENERATE_RUNES_COUNT:
                break

        i += 1
        if i > GENERATE_THEMES_COUNT*3:
            raise Exception("Couldn't generate runes")
    
    for theme, runes in non_duplicate_runes.items():
        display_runes(runes, theme)

    return Runes(ideas = itertools.chain(*non_duplicate_runes.values()))


def generate_runes_for_theme(theme: RuneTheme, count: int) -> Runes:
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": f"I'm making a game in which mundane tasks (such as throwing out trash or doing dishes) can be separated each into multiple runes. Come up with a list of {count} runes that follow the theme \"{theme.name}\" - {theme.description}. The name of each rune should be \"Rune of ...\". Also, for each rune give a brief descriptions what kind of tasks qualify as being \"composed of\" that rune."},
        ],
        response_format=Runes,
    )
    return completion.choices[0].message.parsed

def display_runes(runes: list[Rune], theme: RuneTheme):
    if not DISPLAY_SPOILERS:
        return
    display(Markdown(f"## {theme.name}:\n" + "\n".join(f" - **{rune.name}**: {rune.rune_description} {rune.tasks_composed_of}" for rune in runes)))

runes = generate_runes(themes)

## Nature's Cycle:
 - **Rune of Seed**: Represents the initiation of new life and ideas, symbolizing potential and creativity. Planting seeds, starting a new project, brainstorming ideas, or preparing the soil for gardening.
 - **Rune of Growth**: Symbolizes the nurturing and development of life, representing progress and resilience. Watering plants, fertilizing soil, consistent practice on a skill, or caring for animals.
 - **Rune of Bloom**: Represents flourishing and beauty, marking the peak of life and success in endeavors. Fully blooming flowers, completing a creative project, achieving a goal, or showcasing talents.
 - **Rune of Harvest**: Symbolizes the gathering and receiving of resources, emphasizing the reward of hard work. Collecting fruits, completing tasks, wrapping up projects, or reaping the benefits of previous efforts.
 - **Rune of Decay**: Represents the natural breakdown and recycling processes, symbolizing rejuvenation and rebirth. Composting organic waste, cleaning out old materials, recycling, or clearing out clutter.
 - **Rune of Rest**: Symbolizes the necessary pause for recovery and reflection, marking the end of one cycle before another begins. Taking breaks, enjoying nature, meditating, or practicing self-care.
 - **Rune of Renewal**: Represents the cycle’s reset, embodying transformation and new beginnings following decay. Planning new goals, replanting after a harvest, refreshing spaces, or making resolutions.
 - **Rune of Germination**: Represents the early stages where life begins to emerge from the ground. Watering newly planted seeds, monitoring humidity levels in a greenhouse, or providing warmth to sprouting plants.

## Elemental Forces:
 - **Rune of Grounding**: This rune embodies the stability and strength of the Earth element, promoting a sense of order and structure. Organizing, decluttering, arranging items, sorting through belongings, and grounding routines.
 - **Rune of Flow**: Inspired by the flowing nature of Water, this rune represents tasks that require a fluid motion or rhythm, allowing for natural progression. Washing dishes, watering plants, doing laundry, mopping floors, and taking showers.
 - **Rune of Ignition**: Harnessing the energy of Fire, this rune motivates one to take action and sparks enthusiasm towards completing tasks. Setting goals, motivating oneself to declutter, igniting passion for projects, lighting candles for ambiance, and starting new ventures.
 - **Rune of Elevation**: Reflecting the uplifting qualities of Air, this rune encourages lightness, inspiration, and creativity in mundane activities. Playing music while cleaning, meditating during chores, practicing mindfulness, uplifting conversations while working, and enjoying aesthetic arrangements.
 - **Rune of Purity**: Drawing from the cleansing essence of Water, this rune focuses on purification and freshness in any task. Cleaning surfaces, scrubbing floors, purifying the air (through ventilation or plants), washing linens, and sanitizing spaces.

## Daily Rhythm:
 - **Rune of Dawn**: Symbolizing the start of a new day, this rune embodies the freshness and vitality brought by morning light. Waking up, making the bed, brewing coffee, or preparing a healthy breakfast.
 - **Rune of Morning**: Representing the busyness of the morning routine, this rune signifies productivity and energy levels rising after the dawn. Showering, getting dressed, doing skincare, or packing lunch.
 - **Rune of Noon**: The peak of the day, this rune reflects the height of productivity and the completion of significant tasks. Doing major tasks like working, studying, running errands, or completing household chores.
 - **Rune of Afternoon**: This rune symbolizes a time of reflection and reassessment of goals after the peak productivity wanes. Taking a break, having lunch, tidying up workspaces, or engaging in light reading.
 - **Rune of Evening**: Signifying the winding down of activities, this rune captures the cozy and calming atmosphere as day turns to night. Preparing and eating dinner, watching TV, or spending time with family.
 - **Rune of Dusk**: Marking the transition to nighttime, this rune signifies the slowing down of energy and shift towards relaxation. Cleaning up after dinner, preparing for the next day, or casual conversations.
 - **Rune of Midnight**: This rune symbolizes reflection and introspection as the day comes to a close. Relaxing activities like reading, journaling, or contemplating accomplishments and challenges.

## Generate rune sprites

### Visual concept

In [60]:
class RunePictogram(BaseModel):
    rune: str
    symbol: str

class RunePictograms(BaseModel):
    runes: list[RunePictogram]

class RuneWithPictogram(Rune):
    symbol: str

def generate_runes_pictograms(runes: Runes) -> list[RuneWithPictogram]:
    rune_names = "\n".join([rune.name for rune in runes.ideas])
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": f"What rune-like symbol/pictogram would best fit the following runes? For each rune, reply with a single answer containing max three words. Try to make each rune recognizable and distinct from the other ones. Make all your ideas \"fit\" with one another. Prefer simple symbols over intricate ones. Remember that those are supposed to be font-like characters for runes, not entire pictures.\n\n{rune_names}"},
        ],
        response_format=RunePictograms,
    )
    pictograms = completion.choices[0].message.parsed
    
    pictogram_map = {p.rune: p.symbol for p in pictograms.runes}
    return [
        RuneWithPictogram(**rune.dict(), symbol = pictogram_map.get(rune.name, rune.name))
        for rune in runes.ideas
    ]

def display_runes_pictograms(runes: list[RuneWithPictogram]):
    if not DISPLAY_SPOILERS:
        return
    display(Markdown("\n".join(f" - **{rune.name}**: {rune.symbol}" for rune in runes)))

rune_pictograms = generate_runes_pictograms(runes)
display_runes_pictograms(rune_pictograms)

 - **Rune of Seed**: Tiny Circle
 - **Rune of Growth**: Upward Arrow
 - **Rune of Bloom**: Open Flower
 - **Rune of Harvest**: Grain Bundle
 - **Rune of Decay**: Falling Leaf
 - **Rune of Rest**: Horizontal Line
 - **Rune of Renewal**: Circle Arrow
 - **Rune of Germination**: Sprouting Seed
 - **Rune of Grounding**: Triangle Base
 - **Rune of Flow**: Wavy Line
 - **Rune of Ignition**: Spark Star
 - **Rune of Elevation**: Rising Line
 - **Rune of Purity**: Pure Circle
 - **Rune of Dawn**: Half Sun
 - **Rune of Morning**: Sunrise Rays
 - **Rune of Noon**: Full Sun
 - **Rune of Afternoon**: Sunset Rays
 - **Rune of Evening**: Horizon Line
 - **Rune of Dusk**: Fading Circle
 - **Rune of Midnight**: Dark Circle

### Initial sprite generation

In [69]:
def generate_rune_sprites(runes: list[RuneWithPictogram]):
    path_here = os.popen('powershell.exe -Command "(Get-Item .).FullName"').read().replace("\n", "")
    uid = uuid.uuid4()
    runes_str = base64.b64encode(json.dumps([rune.symbol for rune in runes]).encode("ascii")).decode("ascii")

    command = f'cd {COMFYUI_PATH}; python "{path_here}\image_gen_runes.py" --uuid "{uid}" --runes "{runes_str}"'
    full_command = f'powershell.exe -Command {json.dumps(command)}'
    os.system(full_command)

    commands = []
    for rune in runes:
        for i in range(1, 5):
            command = f'Move-Item -Path "{COMFYUI_PATH}\\output\\rune_{uid}_{rune.symbol}_0000{i}_.png" -Destination "{path_here}\\img\\{rune.encoded()}_{i}.png" -Force'
            commands.append(command)
    os.system(f'powershell.exe -Command {json.dumps("; ".join(commands))}')


def display_raw_rune_sprites(runes: list[RuneWithPictogram]):
    if not DISPLAY_SPOILERS:
        return
    
    full_code = ""
    for rune in runes:
        img_code = "\n".join([f'<img src="img/{rune.encoded()}_{i}.png" style="width: 128px" />' for i in range(1, 5)])
        html_code = f"""
            <div>
                <div style="display: flex; justify-content: space-around;">
                    <span style="font-size: 1.6em; font-weight: 600">{rune.name}</span>
                </div>
                <div style="display: flex; justify-content: space-around;">
                    {img_code}
                </div>
            </div>
        """
        full_code += html_code

    full_code = f"""
        <div style="display: flex; flex-direction: row; flex-wrap: wrap; justify-content: space-around;">
            {full_code}
        </div>
    """
    display(HTML(full_code))

generate_rune_sprites(rune_pictograms)
display_raw_rune_sprites(rune_pictograms)

### Sprite rating

In [106]:
class SpriteRating(BaseModel):
    simplicity: int
    rune_like: int
    aesthetic: int
    resemblance: int
    contains_text: bool

class SpriteRatings(BaseModel):
    top_left: SpriteRating
    top_right: SpriteRating
    bottom_left: SpriteRating
    bottom_right: SpriteRating

class RuneWithSpriteRatings(RuneWithPictogram):
    ratings: SpriteRatings

def select_sprite(rune: RuneWithPictogram) -> RuneWithSpriteRatings:
    response = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": f"The attached image is a 2x2 grid of symbols, each occuppying one quadrant. Rate each of the symbols from 1 to 7 in each of the following categories:\n- simplicity of the design, with 7 being a very simple design, and 1 being overly complex\n- similarity to celtic runes or egyptian hieroglyphs, with 7 meaning it fits well with them, and 1 meaning it doesn't fit at all\n - the aesthetic design of the symbol, with 7 being very pretty and well designed one, and 1 being a very poorly designed one\n - resemblance of {rune.stripped()}, with 7 meaning it can be unmistakenly interpreted as {rune.stripped()}, and 1 meaning it has no correlation with {rune.stripped()}\n- identify if that quadrant of the image contains any text",                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url":  f"data:image/png;base64,{prepare_sprite_grid(rune)}",
                            "detail": "high"
                        },
                    },
                ],
            }
        ],
        response_format=SpriteRatings
    )

    rating = response.choices[0].message.parsed
    rune = RuneWithSpriteRatings(**rune.dict(), ratings=rating)
    return rune
    # sprite_id = {PositionEnum.top_left: 1, PositionEnum.top_right: 2, PositionEnum.bottom_left: 3, PositionEnum.bottom_right: 4}[selected_sprite]
    # sprite = encode_image(PILImage.open(f"img/{rune.encoded()}_{sprite_id}.png"))
    # return RuneWithSprite(**rune.dict(), sprite=sprite)

def prepare_sprite_grid(rune: RuneWithPictogram) -> str:
    images = [PILImage.open(f"img/{rune.encoded()}_{i}.png").resize((128, 128)) for i in range(1, 5)]

    grid_image = PILImage.new("RGBA", (256, 256))
    grid_image.paste(images[0], (0, 0))        # Top-left
    grid_image.paste(images[1], (128, 0))      # Top-right
    grid_image.paste(images[2], (0, 128))      # Bottom-left
    grid_image.paste(images[3], (128, 128))    # Bottom-right

    return encode_image(grid_image)

def encode_image(img: PILImage) -> str:
    buffered = BytesIO()
    img.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode("utf-8")

def display_sprites_ratings(rune: RuneWithSpriteRatings):
    if not DISPLAY_SPOILERS:
        return
    
    ratings = [rune.ratings.top_left, rune.ratings.top_right, rune.ratings.bottom_left, rune.ratings.bottom_right]
    ratings = ["\n".join([f"{k}: {v}" for k, v in r.dict().items()]) for r in ratings]
    img_code = "".join([
        f"""
        <div style="display: flex; flex-direction: row;">
            <img src="img/{rune.encoded()}_{i}.png" style="width: 196px" />
            <div style="white-space: pre; margin-left: 8px; font-size: 16px">{ratings[i-1]}</div>
        </div>
        """
        for i in range(1, 5)
    ])
    full_code = f"""
        <div style="display: flex; justify-content: space-around;">
            <span style="font-size: 1.6em; font-weight: 600">{rune.name}</span>
        </div>
        <div style="display: flex; flex-direction: row; flex-wrap: wrap; justify-content: space-around;">
            {img_code}
        </div>
    """
    display(HTML(full_code))

# def display_rune_sprites(runes: list[RuneWithSprite]):
#     if not DISPLAY_SPOILERS:
#         return
    
#     img_code = "".join([
#         f'<img src="data:image/png;base64, {rune.sprite}" style="width: 128px" />'
#         for rune in runes
#     ])
#     full_code = f"""
#         <div style="display: flex; flex-direction: row; flex-wrap: wrap; justify-content: space-around;">
#             {img_code}
#         </div>
#     """
#     display(HTML(full_code))


runes_with_ratings = [select_sprite(rune) for rune in rune_pictograms]
for rune in runes_with_ratings:
    display_sprites_ratings(rune)